
---

### This 📘 notebook needs to be run from VS Code with [these prerequisites](https://github.com/johnmaeda/SK-Recipes#-prerequisites-to-getting-started-with-the-notebooks-in-this-repo).

How to use this notebook: 

* Just read the text and scroll along until you run into code blocks.
* Code blocks have computer code inside them — hover over the block and you can run the code.
* Run the code by hitting the ▶️ "play" button to the left. If the code runs you'll see a ✔️. If not, you'll get a ❌.
* The output and status of the code block will appear just below itself — you need to scroll down further to see it.
* Sometimes a code block will ask you for input in a hard-to-notice dialog box 👆 at the top of your notebook window. 

---

# S1E5 • Build Chain 🧂🧂🧂🔥 Reactions

When people ask me why Semantic Kernel has its slightly odd name, the simple and truthful answer is that the initial conceiver of SK, Deputy CTO of Microsoft Sam Schillace, has been a long admirer of the simplicity of the UNIX kernel. UNIX was a radical departure in operating systems for its simplicity and flexibility — especially when it came to a concept called "pipes." UNIX commands were easy to string together into a chain reaction like:

`% do-something | do-more-stuff | do-even-more-stuff >> drop-it-all-in-here.txt`

SK lets you do the same thing with the ability to compose functions together as in a chain of functions that take an input that produces an output, and they are connected end-to-end. For example, let's imagine we want to take a bunch of feedback items from customers and understand their painpoints. We can use the `DesignThinkingSkill.Empathize` to give that a go.

## Step 1. Instantiate a 🔥 kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.8.56.1-preview"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Grab the locally stored credentials from the settings.json file. Name the "backend" as "davinci" — assuming that you're using one of the davinci completion models. 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAICompletionBackend("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAICompletionBackend("davinci", model, apiKey, orgId);

## Step 2: Load up a 🧂 skill and 🔥 up the kernel

In [ ]:
using System.Text.Json;
using System.Text.Json.Serialization;

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

// The default input variable 
var input = @"
Customer 1: The power button on my phone is broken. The warranty is still valid.
Customer 2: My display stopped working.
Customer 3: The customer service rep didn't answer my email.
Customer 4: Every time I call customer support I get no answer.
Customer 5: The display screen cracked and it's still under warranty.
Customer 6: My power button fell off the phone. That's ridiculous.
Customer 7: I'm so frustrated with this company.
Customer 8: When I use the power button too much, it stops working.
";

var empathyResult = await kernel.RunAsync(input, skillDT["Empathize"]);

Console.WriteLine(empathyResult);

![](../assets/empathize.png) This is commonly known as the "Empathize" phase of Stanford University's approach to "Design Thinking" with the other phases being: Define, Ideate, Prototype, Test.

## Step 3. 🧂🔥 Turn the empathy insights into problem definitions

We can take this analysis and feed it into a problem-definer skill we have in the `DesignThinkingSkill` toolkit:

In [ ]:

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

var defineResult = await kernel.RunAsync(empathyResult.ToString(), skillDT["Define"]);

Console.WriteLine(defineResult);

## Step 4. 🧂🔥 Brainstorm ideas to address the problems

And if we were in a mood to attempt to solve their problem, we could use the `DesignThinkingSkill.Ideate` function to brainstorm ideas to address the problems as easy versus difficult:

In [ ]:

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

var ideateResult = await kernel.RunAsync(defineResult.ToString(), skillDT["Ideate"]);

Console.WriteLine(ideateResult);

## Step 6. 🧂🧂🧂🔥 Do it all again, but as a chain reaction 

Each step unlocks more power. But what if we could just chain these all together instead of just incrementally process them. That would set a chain reaction in motion from customer support logs all the way to ideas of how to address their concerns. Let's build that:

In [ ]:
using System.Text.Json;
using System.Text.Json.Serialization;

// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the EmpathizeSkill from the Skills Directory
var skillDT = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "DesignThinkingSkill");

var input = @"
Customer 1: The power button on my phone is broken. The warranty is still valid.
Customer 2: My display stopped working.
Customer 3: The customer service rep didn't answer my email.
Customer 4: Every time I call customer support I get no answer.
Customer 5: The display screen cracked and it's still under warranty.
Customer 6: My power button fell off the phone. That's ridiculous.
Customer 7: I'm so frustrated with this company.
Customer 8: When I use the power button too much, it stops working.
";

var result = await kernel.RunAsync(input, skillDT["Empathize"], skillDT["Define"], skillDT["Ideate"]);

Console.WriteLine(result);

Neat, huh? We just connected each of the functions together so the first input goes into the next output, goes into a new input, and sends out a new output, etc.